In [215]:
# Include necessary imports
import os
import torch 
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
import torch.nn.functional as F

In [216]:
# Pre Process Data
def get_pitch_class(note):
    return note % 12

def find_matching_octave_note(df):
    bass_line = df.iloc[3].values
    last_bass_note = bass_line[-1]
    return last_bass_note

def explore_for_lowest_tonic(df, pitch_class):
    bass_notes = df.iloc[3].values  
    matching_notes = [note for note in bass_notes if get_pitch_class(note) == pitch_class]
    if matching_notes:
        return min(matching_notes) 
    else:
        return bass_notes[0]

def detect_tonic(df):
    candidate_note = find_matching_octave_note(df)
    pitch_class = get_pitch_class(candidate_note)
    true_tonic_note = explore_for_lowest_tonic(df, pitch_class)
    return true_tonic_note

def key_transposition(df):
    tonic_note = detect_tonic(df)
    transpose_val = 48 - tonic_note
    df = (df + transpose_val).clip(lower=0, upper=127)
    return df


def normalize_df(df):
    X_std = df / 127
    return X_std

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            if filename != '.ipynb_checkpoints':
                df = pd.read_csv(folder_path + dirname + '/' + filename)
                
                transposed_df = key_transposition(df.transpose())
                normalized_df = normalize_df(transposed_df)
                if dirname == 'test':
                    test.append(normalized_df)
                if dirname == 'train':
                    train.append(normalized_df)
                if dirname == 'valid':
                    validation.append(normalized_df)

# Model

In [291]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=100, n_layers=2, dropout_rate=.3):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True, dropout=dropout_rate)
        self.layer_norms = torch.nn.ModuleList([torch.nn.LayerNorm(hidden_dim) for _ in range(n_layers)])
        self.fc = torch.nn.Linear(hidden_dim, output_size * 128)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device),
                torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(next(self.parameters()).device))

    def forward(self, x, target=None, hidden=None, teacher_forcing=0.5):
        batch_size, seq_len, _ = x.size()

        if hidden is None:
            hidden = self.init_hidden(batch_size)

        lstm_output, hidden = self.lstm(x, hidden)

        for layer_idx in range(self.n_layers):
            lstm_output = self.layer_norms[layer_idx](lstm_output)
        
        lstm_output = self.dropout(lstm_output)
        model_output = self.fc(lstm_output)
        model_output = model_output.view(batch_size, seq_len, 3, 128)

        outputs = []

        for t in range(seq_len):
            model_output_step = model_output[:, t, :, :]  # [batch_size, 3, 128]

            if target is not None and random.random() < teacher_forcing:
                # using teacher forcing
#                 print(F.one_hot((target[:, t, :] * 128).squeeze().to(torch.int64), num_classes=128))
                next_input = F.one_hot((target[:, t, :] * 128).to(torch.int64), num_classes=128).float()  

#                 print(next_input)# [batch_size, 3, 128]
#                 print("teacher forcing next input: ", next_input)
            else:
                # using the model's actual output
#                 pred_indices = model_output_step  # [batch_size, 3, 1]
#                 next_input = F.one_hot((pred_indices).to(torch.int64), num_classes=128).float()  # [batch_size, 3, 128]
                next_input = model_output_step
#                 pred_indices = model_output_step.argmax(dim=-1)  # [batch_size, 3]
#                 next_input = F.one_hot(pred_indices, num_classes=128).float()
#                 print("next input for the model: ", next_input)
#                 print("model output next input: ", next_input)
#             next_input = next_input.view(batch_size, 1, -1)


            outputs.append(next_input.unsqueeze(1))

        output = torch.cat(outputs, dim=1)
        output = output.view(batch_size * seq_len * 3, 128)
#         print("OUTPUT IS", outputs)
        return output, hidden

# Train

In [294]:
def train_model(model, optimizer, criterion, num_epochs):
    model.train()
    for song_index, song in enumerate(train[:5]):
        print(f"Training on song {song_index + 1}")
        
        melody = torch.tensor(song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, song.shape[1], 1)
        harmonies = torch.tensor(song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
#         harmonies_with_zero = torch.zeros(1, song.shape[1], 3)
#         melody_with_empty_harmonies = torch.cat((melody, harmonies_with_zero), dim = -1)
        harmonies_for_loss = harmonies_to_class(harmonies)
        
        hidden = None
        
        for epoch in range(num_epochs):
            optimizer.zero_grad()
            ratio = .9 - (.9 - .1) * (epoch / num_epochs)
            ratio = max(.1, ratio)
            output, hidden = model(x=melody, target=harmonies, hidden=hidden, teacher_forcing=ratio)
#             output = output.reshape(-1, 128)
            harmonies_for_loss = harmonies_for_loss.reshape(-1)
            output = output.view(-1, 128)  # [batch_size * seq_len * 3, 128]
            batch_size, seq_len, _ = melody.size()
            output = output.view(batch_size * seq_len * 3, 128)
#             print("output is ", output)
#             print("harmonies are: ", harmonies)
#             target = harmonies.view(-1).long()
#             print("target is ", harmonies_for_loss)
            loss = criterion(output, harmonies_for_loss)

#             with torch.no_grad():
#                 print(f"Targets    : {harmonies_for_loss[:50]}")
#                 print(f"Predictions: {output[:50].argmax(dim=1)}")
#             print("Output is ", output)
#             print("Harmonies for loss is ", harmonies_for_loss)
#             loss = criterion(output, harmonies_for_loss)
#             loss.requires_grad = True
            loss.backward()
#             for name, param in model.named_parameters():
#                 if param.grad is not None:
#                     print(f"Layer: {name}, Gradient: {param.grad.norm()}")
#                 else:
#                     print(f"Layer: {name}, No gradient found!")
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Song {song_index + 1}, Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
            hidden = tuple(h.detach() for h in hidden)
                    

        model.eval()
        total_val_loss = 0
        

        
        # validation songs
        with torch.no_grad():
#             hidden = None
            for val_song in validation:
                val_melody = torch.tensor(val_song.iloc[0].values.reshape(-1, 1), dtype=torch.float32).unsqueeze(0).reshape(1, val_song.shape[1], 1)
                val_harmonies = torch.tensor(val_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
                val_harmonies = harmonies_to_class(val_harmonies)
#                 val_harmonies_with_zero = torch.zeros(1, val_song.shape[1], 3)
#                 val_melody_with_empty_harmonies = torch.cat((val_melody, val_harmonies_with_zero), dim=-1)
                
                val_output, hidden = model(val_melody, hidden=hidden)
                val_output = val_output.reshape(-1, 128)
                val_harmonies = val_harmonies.reshape(-1)
                
                val_loss = criterion(val_output, val_harmonies)
                with torch.no_grad():
                    print(f"Targets    : {val_harmonies[:50]}")
                    print(f"Predictions: {val_output[:50].argmax(dim=1)}")
                total_val_loss += val_loss.item()
                hidden = tuple(h.detach() for h in hidden)

        average_val_loss = total_val_loss / len(validation)
        print(f"Validation Loss after song {song_index + 1}: {average_val_loss}")

In [295]:
def harmonies_to_class(harmonies):
    harmonies_classes = torch.round(harmonies * 127).long()  
    return harmonies_classes

criterion = torch.nn.CrossEntropyLoss()
num_epochs = 100

model = Model(1, 3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
train_model(model, optimizer, criterion, num_epochs)

Training on song 1
Song 1, Epoch 10/100, Loss: 3.753232955932617
Song 1, Epoch 20/100, Loss: 3.4916906356811523
Song 1, Epoch 30/100, Loss: 3.413393259048462
Song 1, Epoch 40/100, Loss: 3.1057865619659424
Song 1, Epoch 50/100, Loss: 3.1097936630249023
Song 1, Epoch 60/100, Loss: 2.787776231765747
Song 1, Epoch 70/100, Loss: 2.7656819820404053
Song 1, Epoch 80/100, Loss: 2.469306468963623
Song 1, Epoch 90/100, Loss: 2.422447919845581
Song 1, Epoch 100/100, Loss: 2.194556474685669
Targets    : tensor([67, 64, 48, 67, 64, 48, 67, 64, 48, 67, 64, 48, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 74, 67, 59, 74, 67, 59, 74, 67, 59, 74, 67, 59,
        72, 67, 58, 72, 67, 58, 72, 67, 58, 72, 67, 58, 72, 69])
Predictions: tensor([72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67])
Targets    : tensor([63, 60, 60, 63, 60,

Targets    : tensor([60, 57, 53, 60, 57, 53, 60, 57, 51, 60, 57, 51, 61, 58, 49, 61, 58, 49,
        61, 56, 46, 61, 56, 46, 60, 54, 51, 60, 54, 51, 58, 51, 51, 58, 51, 51,
        57, 53, 51, 57, 53, 51, 57, 53, 48, 57, 53, 48, 58, 53])
Predictions: tensor([72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67, 64, 59, 67, 64, 59, 69, 64, 57, 69, 64, 57,
        69, 64, 57, 69, 64, 57, 71, 67, 55, 71, 67, 55, 71, 67, 55, 71, 67, 55,
        72, 67, 48, 72, 67, 48, 70, 67, 48, 70, 67, 48, 69, 65])
Predictions: tensor([72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67,

Targets    : tensor([60, 57, 53, 60, 57, 53, 60, 57, 51, 60, 57, 51, 61, 58, 49, 61, 58, 49,
        61, 56, 46, 61, 56, 46, 60, 54, 51, 60, 54, 51, 58, 51, 51, 58, 51, 51,
        57, 53, 51, 57, 53, 51, 57, 53, 48, 57, 53, 48, 58, 53])
Predictions: tensor([67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48,
        67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48,
        67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67, 64, 59, 67, 64, 59, 69, 64, 57, 69, 64, 57,
        69, 64, 57, 69, 64, 57, 71, 67, 55, 71, 67, 55, 71, 67, 55, 71, 67, 55,
        72, 67, 48, 72, 67, 48, 70, 67, 48, 70, 67, 48, 69, 65])
Predictions: tensor([67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48,
        67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48,
        67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60, 48, 67, 60])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67,

Targets    : tensor([60, 51, 48, 60, 51, 48, 60, 51, 48, 60, 51, 48, 60, 55, 48, 60, 55, 48,
        60, 55, 47, 60, 55, 47, 62, 56, 48, 62, 56, 48, 63, 50, 47, 65, 50, 47,
        63, 51, 48, 63, 51, 48, 59, 53, 50, 59, 53, 50, 60, 55])
Predictions: tensor([72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67])
Targets    : tensor([67, 63, 60, 67, 63, 60, 67, 63, 58, 67, 63, 58, 65, 62, 56, 65, 62, 56,
        65, 60, 56, 65, 60, 56, 67, 62, 55, 67, 62, 55, 67, 60, 55, 67, 60, 55,
        67, 58, 55, 67, 58, 55, 67, 60, 57, 67, 60, 57, 67, 62])
Predictions: tensor([72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67, 60, 72, 67])
Targets    : tensor([63, 60, 48, 63, 60, 48, 65,

Song 4, Epoch 10/100, Loss: 3.6588518619537354
Song 4, Epoch 20/100, Loss: 3.396188259124756
Song 4, Epoch 30/100, Loss: 3.3801958560943604
Song 4, Epoch 40/100, Loss: 3.2330381870269775
Song 4, Epoch 50/100, Loss: 3.115644693374634
Song 4, Epoch 60/100, Loss: 2.7947452068328857
Song 4, Epoch 70/100, Loss: 2.786458730697632
Song 4, Epoch 80/100, Loss: 2.7003324031829834
Song 4, Epoch 90/100, Loss: 2.5150058269500732
Song 4, Epoch 100/100, Loss: 2.444310426712036
Targets    : tensor([67, 64, 48, 67, 64, 48, 67, 64, 48, 67, 64, 48, 72, 67, 60, 72, 67, 60,
        72, 67, 60, 72, 67, 60, 74, 67, 59, 74, 67, 59, 74, 67, 59, 74, 67, 59,
        72, 67, 58, 72, 67, 58, 72, 67, 58, 72, 67, 58, 72, 69])
Predictions: tensor([75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70])
Targets    : tensor([63, 60, 60, 63, 60, 60, 65, 62, 60, 

Targets    : tensor([67, 58, 51, 67, 58, 51, 68, 58, 53, 68, 58, 53, 70, 63, 55, 70, 63, 55,
        70, 63, 55, 70, 63, 55, 68, 63, 56, 68, 63, 56, 68, 63, 56, 68, 63, 56,
        70, 63, 55, 70, 63, 55, 70, 63, 55, 70, 63, 55, 71, 63])
Predictions: tensor([75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70])
Targets    : tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 64, 60, 48, 64, 60, 48, 64, 60, 48, 64, 60, 48,
        65, 60, 53, 65, 60, 53, 65, 60, 53, 65, 60, 53, 65, 60])
Predictions: tensor([75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60,
        75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70, 60, 75, 70])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67,

Targets    : tensor([67, 58, 51, 67, 58, 51, 68, 58, 53, 68, 58, 53, 70, 63, 55, 70, 63, 55,
        70, 63, 55, 70, 63, 55, 68, 63, 56, 68, 63, 56, 68, 63, 56, 68, 63, 56,
        70, 63, 55, 70, 63, 55, 70, 63, 55, 70, 63, 55, 71, 63])
Predictions: tensor([67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64])
Targets    : tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 64, 60, 48, 64, 60, 48, 64, 60, 48, 64, 60, 48,
        65, 60, 53, 65, 60, 53, 65, 60, 53, 65, 60, 53, 65, 60])
Predictions: tensor([67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64])
Targets    : tensor([67, 64, 60, 67, 64, 60, 67,

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, Weights: {param.data}")

# Hyperparameter Tuning

In [ ]:
learning = [0.01, .001]
n_layers= [1,2,3]
hidden_dim = [20, 40, 50]
epochs= [5000, 10000]
best_loss = float('inf')
best_params = {}

for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            for dims in hidden_dim:
                print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch} and hidden_dims={dims}")
                model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer, hidden_dim=dims)
                optimizer = torch.optim.Adam(model.parameters(), lr=LR)
                train_model(model, melody, harmonies, optimizer, criterion, epoch)
                with torch.no_grad():
                    output = model(melody)
                    loss = criterion(output, harmonies)
                    print(f"Final Loss: {loss.item()}")        
                # Keep track of the best model (with lowest loss)
                if loss.item() < best_loss:
                    best_loss = loss.item()
                    best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch, 'hidden_dim': dims}
print("BEST: ", best_params)

In [220]:
def inverse_df(df):
    X_scaled = df * 127
    return X_scaled

def midi_to_note(part):
    print("PART:", part[0])
    result = stream.Part()
    count = 1
    prev = round(part[0])
    for i in range(1, len(part)):
        pitch = part[i]
        curr = round(pitch)
        if curr == prev:
            count += 1
        else:
            this_note = note.Note(prev, quarterLength=count/4)
            result.append(this_note)
            count = 1
        prev = curr
    this_note = note.Note(prev, quarterLength=count/4)
    result.append(this_note)
    return result

# def midi_to_note_harmonies(part):
#     probabilities = torch.softmax(torch.tensor(part), dim=-1).numpy()
# #     print(probabilities)
#     result = stream.Part()
#     count = 1
#     predicted_notes = np.softmax(part, axis=1)
#     print(predicted_notes)
#     prev = predicted_notes[0]
#     for i in range(1, len(part)):
#         curr = predicted_notes[i]
#         if curr == prev:
#             count += 1
#         else:
#             this_note = note.Note(prev, quarterLength=count/4)
#             result.append(this_note)
#             count = 1
#         prev = curr
#     this_note = note.Note(prev, quarterLength=count/4)
#     result.append(this_note)
#     return result

def output_to_sheet_music(melody, result):
    melody = inverse_df(melody).squeeze()
    inversed = result.T
    
    score = stream.Score()
    melody_part = midi_to_note(melody.numpy())
    
    alto_notes = result[0::3].numpy()  # Start at 0, take every 3rd element
    tenor_notes = result[1::3].numpy() # Start at 1, take every 3rd element
    bass_notes = result[2::3].numpy()
    
    print(tenor_notes)
    
    alto_part = midi_to_note(alto_notes)
    tenor_part = midi_to_note(tenor_notes)
    bass_part = midi_to_note(bass_notes)

    score.append(melody_part)
    score.append(alto_part)
    score.append(tenor_part)
    score.append(bass_part)
    score.show('midi')
    score.write('musicxml', 'output.xml')

In [296]:
test_song = test[0]
melody = test_song.iloc[0]

melody = torch.tensor(test_song.iloc[0].values, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
harmonies = torch.tensor(test_song.iloc[1:].values.T, dtype=torch.float32).unsqueeze(0)
harmonies_with_zero = torch.zeros(1, test_song.shape[1], 3)

model_input = torch.cat((melody, harmonies_with_zero), dim = -1)
result, _ = model(x=melody)
print(result)
predictions = result.argmax(dim=-1)
print("Predictions: ", predictions)
output_to_sheet_music(melody, predictions)

tensor([[-5.8985, -6.2813, -6.5655,  ..., -6.6102, -5.3822, -6.3479],
        [-7.2255, -5.4450, -6.7801,  ..., -5.5966, -4.9602, -6.5642],
        [-6.3287, -6.2678, -6.3982,  ..., -6.4567, -6.5853, -5.2883],
        ...,
        [-5.9808, -6.2965, -6.1660,  ..., -5.8686, -5.3908, -5.8441],
        [-6.5488, -5.0975, -5.8044,  ..., -5.7963, -5.2072, -5.7273],
        [-6.4455, -5.7424, -5.9553,  ..., -5.8655, -6.0974, -5.5714]],
       grad_fn=<ViewBackward0>)
Predictions:  tensor([71, 60, 48, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55, 67, 64, 55,
        67, 64, 55, 67, 64, 55, 67, 64, 

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??

In [ ]:
# real one and generated compare
# train on all songs + test on a different song
# measure the test loss not just the training loss

In [ ]:
# weights get zeroed out
# vanishing gradient
# not adjusting the weights; gradient is zero?